In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from retrograd.engine import Value
from retrograd.neural import Neuron, Layer, MLP

In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
from sklearn.datasets import make_moons, make_blobs
X,y = make_moons(n_samples = 100, noise = 0.1)

plt.figure(figsize=(5,5))
plt.scatter(X[:,0],X[:,1],c=y,s=20,cmap='jet')

In [ ]:
#init neural network
model = MLP(2,[8,8,1])
print(f'Number of params: {len(model.get_parameters())}')

In [ ]:
#minibatch loss
def loss(batch_size=None):
  if batch_size is None:
    Xb,yb = X,y
  else:
    ri = np.random.permutation(X.shape[0])[:batch_size]
    Xb,yb = X[ri],y[ri]
  inputs = [list(map(Value,xrow)) for xrow in Xb]

  #forward
  scores = list(map(model,inputs))

  probs = [s.sigmoid() for s in scores]

  # binary cross entropy loss
  eps = 1e-15
  losses = [-((yr*((yp+eps).log()))+((1-yr)*((1-yp+eps).log()))) for yr,yp in zip(yb,probs)]
  data_loss = sum(losses) * (1.0 / len(losses)) # avg loss

  total_loss= data_loss

  # accuracy
  accuracy = [yr == np.round(yp.data) for yr,yp in zip(yb,probs)]
  return total_loss, sum(accuracy)/len(accuracy)


In [ ]:
#training
for k in range(100):
  #forward
  total_loss,acc = loss()

  #backward
  model.zero_grad()
  total_loss.backward()

  #update
  learning_rate=1.0 - 0.9*k/100 # learning rate decay
  for p in model.get_parameters():
    p.data -= learning_rate * p.grad

  if k%10 ==0:
    print(f'Step {k} | Loss: {total_loss.data}, accuracy: {acc*100}%')
print(f'Step {k} | Loss: {total_loss.data}, accuracy: {acc*100}%')

In [ ]:
# Visualize decision boundary
h = 0.1  # Smaller step for smoother boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Get predictions for each point in the mesh
mesh_points = np.c_[xx.ravel(), yy.ravel()]
mesh_inputs = [list(map(Value, point)) for point in mesh_points]
mesh_scores = [model(inputs) for inputs in mesh_inputs]
mesh_probs = [s.sigmoid()for s in mesh_scores]
mesh_predictions = np.array([score.data for score in mesh_probs])
mesh_predictions = mesh_predictions.reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, mesh_predictions, levels=10, alpha=0.8, cmap='RdYlBu')
plt.colorbar(label='Prediction Probability')
plt.contour(xx, yy, mesh_predictions, levels=[0.5], colors='black', linestyles='--', linewidths=2)

# Plot original data points
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap='RdYlBu', edgecolors='black')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Neural Network Decision Boundary')
plt.show()